In [1]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import os

In [2]:
data = pd.read_csv("./data/driving_log.csv")
X = data[['center', 'left', 'right']].values
y = data['steering'].values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=0)

In [3]:
def choose(center, left, right, angle):
    choice = np.random.choice(3)
    if choice == 0:
        return image(left), angle + 0.2
    elif choice == 1:
        return image(right), angle - 0.2
    return image(center), angle


def image(file):
    return mpimg.imread("./data/"+file.strip())



def translate(image, angle, rangex, rangey):
    transx = rangex * (np.random.rand() - 0.5)
    transy = rangey * (np.random.rand() - 0.5)
    angle += transx * 0.002
    transm = np.float32([[1, 0, transx], [0, 1, transy]])
  
    height, width = image.shape[:2]
    image = cv2.warpAffine(image, transm, (width, height))
    return image, angle


def flip(image, angle):
  
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
        angle = - angle
    return image, angle

def preprocess(image):
    image = cv2.resize(image, (320, 160), cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    return image

def shadow(image):
    x1, y1 = 320 * np.random.rand(), 0
    x2, y2 = 320 * np.random.rand(), 160
    xm, ym = np.mgrid[0:160, 0:320]
    mask = np.zeros_like(image[:, :, 1])
    mask[(ym - y1) * (x2 - x1) - (y2 - y1) * (xm - x1) > 0] = 1
    cond = mask == np.random.randint(2)
    s_ratio = np.random.uniform(low=0.2, high=0.5)
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    hls[:, :, 1][cond] = hls[:, :, 1][cond] * s_ratio
    return cv2.cvtColor(hls, cv2.COLOR_HLS2RGB)

def augument(center, left, right, angle, rangex=100, rangey=10):
    image, angle = choose( center, left, right,angle)
    image, angle = flip(image, angle)
    image, angle = translate(image, angle, rangex, rangey)
    image = shadow(image)
    image = brightness(image)
    return image, angle



def brightness(image):
    # HSV (Hue, Saturation, Value) is also called HSB ('B' for Brightness).
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    ratio = 1.0 + 0.4 * (np.random.rand() - 0.5)
    hsv[:,:,2] =  hsv[:,:,2] * ratio
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

def generator(image_paths, angles, batch_size, training):
    images = np.empty([batch_size,160,320,3])
    steers = np.empty(batch_size)
    while True:
        i = 0
        for index in np.random.permutation(image_paths.shape[0]):
            center, left, right = image_paths[index]
            angle = angles[index]
            # argumentation
            if training and np.random.rand() < 0.6:
                image, angle = augument(center, left, right, angle)
            else:
                image = image(center) 
            # add the image and steering angle to the batch
            images[i] = preprocess(image)
            steers[i] = angle
            i = i + 1
            if i == batch_size:
                break
        yield images, steers

In [8]:
import keras
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Cropping2D,Flatten,Dense, Convolution2D, MaxPooling2D, Dropout, Lambda
from keras.optimizers import Adam

In [9]:
model = Sequential()
model.add(Lambda(lambda x: x/255.0-0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(24, (5, 5), activation='relu', subsample=(2, 2)))
model.add(Convolution2D(36, (5, 5), activation='relu', subsample=(2, 2)))
model.add(Convolution2D(48, (5, 5), activation='relu', subsample=(2, 2)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_2 (Cropping2D)    (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 31, 158, 24)       1824      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 77, 36)        21636     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 5, 37, 48)         43248     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 3, 35, 64)         27712     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 1, 33, 64)         36928     
__________

/home/nani/.conda/envs/car/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), strides=(2, 2), activation="relu")`
/home/nani/.conda/envs/car/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), strides=(2, 2), activation="relu")`
/home/nani/.conda/envs/car/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), strides=(2, 2), activation="relu")`


In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='auto')

model.compile(loss='mse', optimizer=Adam(lr=1e-3))
model.fit_generator(generator( X_train, y_train, 64, True), 20000,
validation_data=generator(X_valid, y_valid, 64, False), validation_steps=len(X_valid)/64, epochs=10, verbose = 1)


Epoch 1/10
20000/20000 [==============================] - 6012s 301ms/step - loss: 208.4458 - val_loss: 35.3267
Epoch 2/10
10102/20000 [==============>...............] - ETA: 1:15:11 - loss: 155.4062

In [7]:
model.save("model.h5")